In [55]:
import uuid
import pandas as pd
import numpy as np
from datetime import datetime

from project_huishoudboekje.database import connect_to_database
from project_huishoudboekje.config import GeneralSettings

In [2]:
df = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions_test.xlsx')

In [3]:
df.head()

,TRANS_ID,DATE,SOURCE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,CATEGORY,ANALYSE_IND,GROUP
0,e1d07890-e896-48cb-bebb-618582c050f9,2025-08-11,ASN Bank,-,debet,test2,20.00,Overige aankopen,1,Aankopen
1,572129d3-2d81-4b86-9a17-c34fdcec7584,2025-08-10,ASN Bank,-,credit,test,-10.00,Overige aankopen,1,Aankopen
2,fa393fc6-264a-44bb-ae0f-c2f8d1ab374c,2025-07-31,ASN Bank,OVS,credit,Zalando Payments GmbH - 011113919304-DE3410070...,85.49,Kleding en schoenen,1,Aankopen
3,233372ca-25e1-4a8b-969d-88886b84a16e,2025-07-31,ASN Bank,-,debet,Gezonderwinkelen.nl via MultiSafepay - 1221335...,8.36,Baby artikelen,1,Aankopen
4,2f2575a4-0a35-4109-87c1-6780bf772834,2025-07-31,ASN Bank,EIC,debet,VOGELBESCHERMING NEDERLAND - Europese incasso:...,5.00,Overige goede doelen,1,Giften


In [4]:
datetime.now().timestamp()

1754939150.916451

In [9]:
df['TS_CHANGED'] = datetime.now().timestamp()

In [6]:
cursor, conn = connect_to_database()

In [7]:
df = df[~df.TRANS_ID.duplicated()].copy()

In [10]:
df.rename(
        columns={'GROUP': 'GROUPLEVEL'}).to_sql(name='transactions',
                                             con=conn,
                                             if_exists='append',
                                             index=False)

conn.commit()

### Fill production database

In [2]:
df_2022 = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions_2022.xlsx')

In [3]:
df_2023 = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions_2023.xlsx')

In [4]:
df_2024 = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions_2024.xlsx')

In [5]:
df_2025 = pd.read_excel(GeneralSettings.project_path / 'data/processed/transactions.xlsx')

In [39]:
df_prod = pd.concat([df_2022, df_2023, df_2024, df_2025], ignore_index=True)

In [40]:
df_prod.shape

(5156, 29)

In [41]:
df_prod_keep = df_prod[['TRANS_ID', 'DATE',	'SOURCE', 'TRANSACTION_TYPE', 'FINANCIAL_TYPE', 'PARTY', 'AMOUNT', 'CATEGORY', 'ANALYSE_IND', 'GROUP']].copy()

In [42]:
df_prod_keep.sample(n=5)

,TRANS_ID,DATE,SOURCE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,CATEGORY,ANALYSE_IND,GROUP
1691,3fe026a2-b9f6-4904-bb16-cea3c6cd6447,2023-09-22,ASN Bank,EIC,debet,Sportcity - sportcity - Membership,11.25,Sportschool,1.0,Leven en entertainment
529,2edfc824-6561-44ca-8b10-95977a9558b0,2022-08-23,Rabobank,sb,credit,Centrale Salarisadministratie - BETALINGSPERIO...,3958.85,Salaris Rob,1.0,Inkomsten
4154,3e114adb-a9ae-4647-9556-f0b287984826,2024-01-12,ASN Bank,IDE,debet,Stichting Open Doors by Buckaroo - pi84s76wz8l...,178.20,Christelijke goede doelen,1.0,Giften
786,daaed1db-5bc0-496a-9850-2ad105daefbd,2022-07-02,Rabobank,-,debet,IKEA Amersfoort AMERSFOORT,11.29,Bruiloft,1.0,Overig
1615,15e46907-4f9c-4aa6-8088-378ecae86414,2023-10-15,ASN Bank,IOS,credit,AAB INZ TIKKIE - Tikkie ID 000724333132,19.89,Uitjes,1.0,Leven en entertainment


In [57]:
df_prod_keep['TRANS_ID'] = df_prod_keep['TRANS_ID'].astype(str).apply(lambda x: uuid.uuid4() if ((x == '') or (x is None) or (x == 'nan')) else x)

In [44]:
df_prod_keep.dtypes

TRANS_ID                    object
DATE                datetime64[ns]
SOURCE                      object
TRANSACTION_TYPE            object
FINANCIAL_TYPE              object
PARTY                       object
AMOUNT                     float64
CATEGORY                    object
ANALYSE_IND                float64
GROUP                       object
dtype: object

In [51]:
df_prod_keep['TS_CHANGED'] = datetime.now().timestamp()

In [46]:
print(df_prod_keep.shape)
print(df_prod_keep[~df_prod_keep.TRANS_ID.duplicated()].shape)

(5156, 11)
(3104, 11)


In [58]:
df_prod_keep[df_prod_keep.TRANS_ID.duplicated()]

,TRANS_ID,DATE,SOURCE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,CATEGORY,ANALYSE_IND,GROUP,TS_CHANGED
114,4597d02e-72bc-4fcf-9040-6b48276ac5ec,2022-12-01,ASN Bank,EIC,debet,NATIONALE-NEDERLANDEN VERZEKEREN - INCASSO ZEK...,14.74,Opstalverzekering,1.0,Verzekeringen,1.755328e+09
157,c0dc4c60-0f71-4850-99dc-d005b095f6ed,2022-11-23,ASN Bank,OVS,credit,Centrale Salarisadministratie - BETALINGSPERIO...,50.00,Thuiswerkvergoeding,1.0,Inkomsten,1.755328e+09
223,cf202032-b639-4d89-88b4-f6988e3993a7,2022-11-04,ASN Bank,EIC,debet,NATIONALE-NEDERLANDEN VERZEKEREN - INCASSO ZEK...,14.74,Opstalverzekering,1.0,Verzekeringen,1.755328e+09
348,20e2b25e-fc08-4b94-b2c8-257b3520fdc8,2022-10-03,Rabobank,ei,debet,NATIONALE-NEDERLANDEN VERZEKEREN - INCASSO ZEK...,14.74,Opstalverzekering,1.0,Verzekeringen,1.755328e+09
402,3729a34f-a7e2-48e3-af3d-4dd3a58a6a23,2022-09-23,Rabobank,sb,credit,Centrale Salarisadministratie - BETALINGSPERIO...,3958.85,Salaris Rob,1.0,Inkomsten,1.755328e+09
486,6c4a4fc4-6426-4837-9372-7987ffe61584,2022-09-01,Rabobank,ei,debet,NATIONALE-NEDERLANDEN VERZEKEREN - INCASSO ZEK...,14.74,Opstalverzekering,1.0,Verzekeringen,1.755328e+09
1600,b741a02c-f7b5-4156-8f05-75db0cd4fd87,2023-10-23,ASN Bank,OVS,credit,Centrale Salarisadministratie - BETALINGSPERIO...,50.00,Thuiswerkvergoeding,1.0,Inkomsten,1.755328e+09
1655,33bf1f38-1a0e-4c97-8c6a-8840d127340c,2023-10-02,Rabobank,ei,debet,InShared Nederland B.V. - Factuur 2023/09 nr.,6.09,Inboedelverzekering,1.0,Verzekeringen,1.755328e+09
1696,4acd8f06-e2e8-4016-a8c2-0929432dfe5d,2023-09-22,ASN Bank,OVS,credit,Centrale Salarisadministratie - BETALINGSPERIO...,4277.62,Salaris,1.0,Inkomsten,1.755328e+09
1760,011f5fd8-a3f3-4063-a9e0-0a88c5fd7252,2023-09-01,Rabobank,ei,debet,InShared Nederland B.V. - Factuur 2023/08 nr.,6.09,Inboedelverzekering,1.0,Verzekeringen,1.755328e+09


In [28]:
df_prod_keep[df_prod_keep.TRANS_ID == '3729a34f-a7e2-48e3-af3d-4dd3a58a6a23']

,TRANS_ID,DATE,SOURCE,TRANSACTION_TYPE,FINANCIAL_TYPE,PARTY,AMOUNT,CATEGORY,ANALYSE_IND,GROUP,TS_CHANGED
401,3729a34f-a7e2-48e3-af3d-4dd3a58a6a23,2022-09-23,Rabobank,sb,credit,Centrale Salarisadministratie - BETALINGSPERIO...,50.00,Thuiswerkvergoeding,1.0,Inkomsten,1.755327e+09
402,3729a34f-a7e2-48e3-af3d-4dd3a58a6a23,2022-09-23,Rabobank,sb,credit,Centrale Salarisadministratie - BETALINGSPERIO...,3958.85,Salaris Rob,1.0,Inkomsten,1.755327e+09


In [59]:
df_prod_keep.iloc[4971, 0]

UUID('9eb45bec-1b73-4eec-81dc-08ddf9437728')

In [60]:
cursor, conn = connect_to_database()

In [63]:
df_prod_keep.rename(
        columns={'GROUP': 'GROUPLEVEL'}).astype({'TRANS_ID': str}).to_sql(name='transactions',
                                             con=conn,
                                             if_exists='append',
                                             index=False)

conn.commit()

In [64]:
cursor.close()
conn.close()